In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [5]:
df['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [6]:
X=df.drop(['RowNumber',	'CustomerId',	'Surname','Exited'],axis=1).to_numpy()
y=df['Exited'].to_numpy()

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [8]:
le =LabelEncoder()
le.fit(['Male','Female'])
X[:,2]=le.transform(X[:,2])

ct=ColumnTransformer(transformers=[
    ('encoder',OneHotEncoder(),[1])],
    remainder='passthrough'
)
X=ct.fit_transform(X)

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y)

In [12]:
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [13]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=6,activation='relu'))
model.add(tf.keras.layers.Dense(units=12,activation='relu'))
model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

2025-02-12 11:40:37.743281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-12 11:40:38.011087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-12 11:40:38.011118: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-02-12 11:40:40.185614: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-

In [14]:
model.fit(X_train_scaled,y_train,batch_size=32,epochs=50)

Epoch 1/50
250/250 [==============================] - 2s 2ms/step - loss: 0.6117 - accuracy: 0.6789
Epoch 2/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4905 - accuracy: 0.7962
Epoch 3/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4767 - accuracy: 0.7961
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4658 - accuracy: 0.7972
Epoch 5/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4536 - accuracy: 0.8039
Epoch 6/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4390 - accuracy: 0.8085
Epoch 7/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4238 - accuracy: 0.8155
Epoch 8/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4093 - accuracy: 0.8189
Epoch 9/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3966 - accuracy: 0.8256
Epoch 10/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3856 - accuracy: 0.8338

In [15]:
preds=model.predict(X_test_scaled)
y_pred = (preds > 0.5)
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

63/63 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1593
           1       0.74      0.47      0.58       407

    accuracy                           0.86      2000
   macro avg       0.81      0.72      0.75      2000
weighted avg       0.85      0.86      0.85      2000

